In [ ]:
pip install ipywidgets

In [ ]:
pip install pandas

In [14]:
import cv2
import csv
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
from pathlib import Path

# Video path and output CSV path
video_path = 'data/videos/test0_toy_lecture.mp4'
output_csv = 'output/txt/annotations.csv'
sampling_frequency = 1  # seconds between samples

# Prepare CSV file
if not Path(output_csv).exists():
    with open(output_csv, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Frame', 'Count', 'Scene change'])  # Headers: frame number, label count, and label

# Load video
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_increment = fps / sampling_frequency

# Widgets
button_change = widgets.Button(description="Scene change")
button_no_change = widgets.Button(description="No scene change")
button_undo = widgets.Button(description="Undo")

# Initialize frame position to the last frame minus increment
total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
frame = total_frames - (total_frames % frame_increment)
count = 0
 

# Define button callback functions
def log_and_next(label):
    global frame
    global count
    global frame_increment

    # Log the label and frame number
    with open(output_csv, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([int(frame + frame_increment), count, label])
        count += 1

    # Move to next frame
    display_buttons()
    display_frame()

def on_change_button_clicked(b):
    log_and_next('True')

def on_no_change_button_clicked(b):
    log_and_next('False')

def on_undo_button_clicked(b):
    global frame
    global count
    global frame_increment
    count -= 1
    frame += 2 * frame_increment
    delete_row_from_csv(count)
    display_buttons()
    display_frame()

def delete_row_from_csv(row_index):
    """Deletes a row from a CSV file.

    Args:
        csv_file: The path to the CSV file.
        row_index: The index of the row to delete (starting from 0).
    """

    with open(output_csv, 'r') as file:
        reader = csv.reader(file)
        rows = list(reader)

    del rows[row_index]

    with open(output_csv, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(rows)

# Attach event handlers
button_change.on_click(on_change_button_clicked)
button_no_change.on_click(on_no_change_button_clicked)
button_undo.on_click(on_undo_button_clicked)

# Display frame
def display_frame():
    global frame
    global count
    global frame_increment

    # Set current frame
    frame -= frame_increment  # Decrement frame by frame_increment
    # Check for end of video
    if frame < 0:
        # Log the label and frame number
        with open(output_csv, 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([int(frame + frame_increment), count, 'True'])
        print("End of Video")
        cap.release()
        return
    cap.set(cv2.CAP_PROP_POS_FRAMES, int(frame))
    ret_current, current_frame_image = cap.read()
    if not ret_current:
        print("Failed to load current frame")
        return
    
    # Set last frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, int(frame + frame_increment))
    ret_last, last_frame_image = cap.read()
    if not ret_last:
        print("Failed to load previous frame")
        return

    # Convert frames to RGB for display
    current_frame_rgb = current_frame_image 
    last_frame_rgb = last_frame_image

    # Display frames using ipywidgets.Image
    print("Scene change if the top frame is different from the bottom frame")
    print("Top image frame: " + str(int(frame)))
    current_frame_widget = widgets.Image(
        value=cv2.imencode('.jpg', current_frame_rgb)[1].tobytes(),
        format='jpg',
        width='45%',
        height='45%',
        min_width = '300px'
    )
    prev_frame = frame + frame_increment
    print("Bottom image frame: " + str(int(prev_frame)))
    last_frame_widget = widgets.Image(
        value=cv2.imencode('.jpg', last_frame_rgb)[1].tobytes(),
        format='jpg',
        width='45%',
        height='45%',
    )

    display(current_frame_widget, last_frame_widget)

# Display buttons
def display_buttons():
    clear_output(wait=True)
    display(widgets.HBox([button_change, button_no_change, button_undo]))

# Start the frame display
display_buttons()
display_frame()

Scene change if the top frame is different from the bottom frame
Top image frame: 775
Bottom image frame: 800


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…